In [ ]:
Task 2: Deep Learning on Raw Inertial Sensor Data

In [6]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define dataset paths
train_dir = r"C:\Users\vedant\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\train"
test_dir = r"C:\Users\vedant\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\test"

# Load raw sensor data
def load_raw_sensor_data(dir_path, files):
    data = []
    for file in files:
        file_path = f"{dir_path}/Inertial Signals/{file}"
        data.append(np.loadtxt(file_path))
    return np.stack(data, axis=-1)  # Shape: (samples, timesteps, features)

# Define signal file names
signal_files = [
    "body_acc_x_", "body_acc_y_", "body_acc_z_",
    "body_gyro_x_", "body_gyro_y_", "body_gyro_z_",
    "total_acc_x_", "total_acc_y_", "total_acc_z_"
]

# Load train and test data
X_train_raw = load_raw_sensor_data(train_dir, [f"{s}train.txt" for s in signal_files])
X_test_raw = load_raw_sensor_data(test_dir, [f"{s}test.txt" for s in signal_files])

# Load labels
y_train_raw = np.loadtxt(f"{train_dir}/y_train.txt").astype(int) - 1
y_test_raw = np.loadtxt(f"{test_dir}/y_test.txt").astype(int) - 1

# Convert labels to one-hot encoding 
num_classes = len(np.unique(y_train_raw))
y_train_one_hot = tf.keras.utils.to_categorical(y_train_raw, num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test_raw, num_classes)

# Input shape for models
input_shape = X_train_raw.shape[1:]  # (timesteps, features)


In [7]:
# Define CNN Model
def build_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
        tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Training
model_cnn = build_cnn_model()
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# evaluation
model_cnn.fit(X_train_raw, y_train_one_hot, validation_data=(X_test_raw, y_test_one_hot), epochs=10, batch_size=32)

# Prediction and calculation
y_pred_cnn = model_cnn.predict(X_test_raw).argmax(axis=1)
cnn_metrics = {
    "Accuracy": accuracy_score(y_test_raw, y_pred_cnn),
    "Precision": precision_score(y_test_raw, y_pred_cnn, average='weighted'),
    "Recall": recall_score(y_test_raw, y_pred_cnn, average='weighted'),
    "F1 Score": f1_score(y_test_raw, y_pred_cnn, average='weighted')
}

print("CNN Metrics:", cnn_metrics)


C:\Users\vedant\Documents\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.6880 - loss: 0.7642 - val_accuracy: 0.8633 - val_loss: 0.4476
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9189 - loss: 0.2191 - val_accuracy: 0.8663 - val_loss: 0.5957
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9397 - loss: 0.1605 - val_accuracy: 0.8602 - val_loss: 0.7325
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9443 - loss: 0.1450 - val_accuracy: 0.9141 - val_loss: 0.4578
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9474 - loss: 0.1322 - val_accuracy: 0.9019 - val_loss: 0.6534
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.9499 - loss: 0.1170 - val_accuracy: 0.8992 - val_loss: 0.5557
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9451 - loss: 0.1328 - val_accuracy: 0.9128 - val_loss: 0.6466
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.9475 - loss: 0.1198 - val_acc

In [8]:
# Define MLP Model
def build_mlp_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Training
model_mlp = build_mlp_model()
model_mlp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# evaluation
model_mlp.fit(X_train_raw, y_train_one_hot, validation_data=(X_test_raw, y_test_one_hot), epochs=10, batch_size=32)

# Prediction and calculation
y_pred_mlp = model_mlp.predict(X_test_raw).argmax(axis=1)
mlp_metrics = {
    "Accuracy": accuracy_score(y_test_raw, y_pred_mlp),
    "Precision": precision_score(y_test_raw, y_pred_mlp, average='weighted'),
    "Recall": recall_score(y_test_raw, y_pred_mlp, average='weighted'),
    "F1 Score": f1_score(y_test_raw, y_pred_mlp, average='weighted')
}

print("MLP Metrics:", mlp_metrics)


Epoch 1/10


C:\Users\vedant\Documents\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5396 - loss: 1.1628 - val_accuracy: 0.8140 - val_loss: 0.6290
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8029 - loss: 0.5337 - val_accuracy: 0.8687 - val_loss: 0.4580
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8780 - loss: 0.3537 - val_accuracy: 0.8697 - val_loss: 0.4281
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8924 - loss: 0.2889 - val_accuracy: 0.8670 - val_loss: 0.4673
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9037 - loss: 0.2860 - val_accuracy: 0.8826 - val_loss: 0.3789
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9105 - loss: 0.2400 - val_accuracy: 0.8802 - val_loss: 0.5861
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9192 - loss: 0.2238 - val_accuracy: 0.8972 - val_loss: 0.3931
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9288 - loss: 0.1978 - val_accuracy: 0.9002 - val

In [9]:
# Define LSTM Model
def build_lstm_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Training
model_lstm = build_lstm_model()
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# evaluation
model_lstm.fit(X_train_raw, y_train_one_hot, validation_data=(X_test_raw, y_test_one_hot), epochs=10, batch_size=32)

# Prediction and calculation
y_pred_lstm = model_lstm.predict(X_test_raw).argmax(axis=1)
lstm_metrics = {
    "Accuracy": accuracy_score(y_test_raw, y_pred_lstm),
    "Precision": precision_score(y_test_raw, y_pred_lstm, average='weighted'),
    "Recall": recall_score(y_test_raw, y_pred_lstm, average='weighted'),
    "F1 Score": f1_score(y_test_raw, y_pred_lstm, average='weighted')
}

print("LSTM Metrics:", lstm_metrics)


C:\Users\vedant\Documents\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 45s 169ms/step - accuracy: 0.4587 - loss: 1.2752 - val_accuracy: 0.6912 - val_loss: 0.8132
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 40s 164ms/step - accuracy: 0.7455 - loss: 0.6369 - val_accuracy: 0.7903 - val_loss: 0.5792
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 40s 161ms/step - accuracy: 0.8630 - loss: 0.4258 - val_accuracy: 0.8649 - val_loss: 0.4466
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 41s 159ms/step - accuracy: 0.9243 - loss: 0.2309 - val_accuracy: 0.8599 - val_loss: 0.5207
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 42s 162ms/step - accuracy: 0.9411 - loss: 0.1794 - val_accuracy: 0.8911 - val_loss: 0.3909
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 41s 162ms/step - accuracy: 0.9431 - loss: 0.1716 - val_accuracy: 0.8823 - val_loss: 0.4970
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 38s 166ms/step - accuracy: 0.9381 - loss: 0.1677 - val_accuracy: 0.8636 - val_loss: 0.4772
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 38s 165ms/step - accuracy: 0.9404 - loss: 0

In [11]:
# Comparison function
def compare_models(cnn_metrics, mlp_metrics, lstm_metrics):
   comparison = {
        'Model': ['CNN', 'MLP', 'LSTM'],
        'Accuracy': [cnn_metrics['Accuracy'], mlp_metrics['Accuracy'], lstm_metrics['Accuracy']],
        'Precision': [cnn_metrics['Precision'], mlp_metrics['Precision'], lstm_metrics['Precision']],
        'Recall': [cnn_metrics['Recall'], mlp_metrics['Recall'], lstm_metrics['Recall']],
        'F1 Score': [cnn_metrics['F1 Score'], mlp_metrics['F1 Score'], lstm_metrics['F1 Score']]
    }
    
    import pandas as pd
    comparison_df = pd.DataFrame(comparison)
    return comparison_df
comparison_results = compare_models(cnn_metrics, mlp_metrics, lstm_metrics)

print(comparison_results)


  Model  Accuracy  Precision    Recall  F1 Score
0   CNN  0.908042   0.912859  0.908042  0.908155
1   MLP  0.912453   0.913602  0.912453  0.912076
2  LSTM  0.900238   0.902387  0.900238  0.899607


In [ ]:
### Model Performance Summary 

In this task,evaluation of three deep learning models for human activity recognition were done.

- MLP achieved the highest accuracy of 91.25%, along with the best precision and recall values.

- CNN followed closely with an accuracy of 90.80% along with decent precision,recall value and F1 Score.

- LSTM had the lowest accuracy at 90.02% with lowest precision,recall value and F1 Score.

### Conclusion:
Based on the results, MLP stands out as the most effective model for this task.

In [ ]:
### Model Performance Comparison: Task 1 (Machine Learning) vs Task 2 (Deep Learning)

- Task 1 Models: Logistic Regression stands out as the best performer in the classic machine learning models, with an impressive accuracy of 95.49%, while Random Forest is a strong second.
- Task 2 Models: MLP performed best among the deep learning models, with 91.25% accuracy, though it falls behind the logistic regression from Task 1.
- Overall Comparison: Classic machine learning models, especially Logistic Regression and Random Forest, generally outperformed the deep learning models in terms of accuracy, precision, and recall for this human activity recognition task.
